In [1]:
from datetime import datetime
from typing import Tuple, List, Union, Dict, Any
import json
import sys
import os
import uuid
import random

# Include subpath as module path
module_path = os.path.abspath(os.path.join(".."))
if module_path not in sys.path:
    sys.path.append(module_path)

print("Module path: ", module_path)

from stream_handler import setup
setup()

from backend.typealias import CoordinatePair, Sequence, Token, Pattern
import backend.datastructures as ds 
import backend.utils as utils
import backend.plotutils as plotutils
import backend.staypointdetector as spdetector
import backend.geohashsequenceinterpolator as ip
import backend.datapointstreamhandler as shandler
import backend.xchanges as xchanges
import backend.trajectorysegmentor as segmentor 
import backend.dataframeinterfaces as dfinterfaces
import backend.sequencescorer as seqscorer
import backend.disorientationdetectors as detectors
import backend.detectorevaluator as de

import utm

from shapely.ops import unary_union
from shapely.geometry import LineString, Polygon, Point


from django.conf import settings

import backend.models as django_models
import backend.managers as django_managers

import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.lines import Line2D
from matplotlib.dates import DateFormatter

import pandas as pd
import numpy as np
import seaborn as sns
import msgpack
import folium
import geohash_hilbert as ghh

# Set the following flag such that we are able to run Django inside a jupyter notebook:
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

# Settings
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['figure.dpi'] = 100 # 200 e.g. is really fine, but slower


bits_per_char = 2
precision = 20
center = [55.3923666589426, 10.386784608945067]
user = "d1b0c137-1cd0-4430-ad88-4dc24c88f3ad" 

Module path:  /home/nicklasxyz/Desktop/traj-dirs/traj-miner-final/RealtimeTrajectoryDataMining/rtdm


In [2]:
def plot_sequence(
    sequence: Sequence,
    bits_per_char: int,
    color: str,           # Stroke color
    opacity: float = 1.0, # Stroke opacity
    weight: float = 1.0,  # Stroke width in pixels
    center: Union[None, CoordinatePair] = None,
    map_: Union[None, folium.Map] = None,
    ) -> folium.Map:
    if map_ is None:# and not center is None:
        map_ = plotutils.setup_map(center = center)
    for geohash in sequence:
        obj = ghh.rectangle(geohash, bits_per_char = bits_per_char)
        bbox_rectangle = obj["geometry"]["coordinates"][0]
        swapped_coords = [(v[1], v[0]) for v in bbox_rectangle]
        folium.PolyLine(
            swapped_coords,
            color = color,
            weight = weight,
            opacity = opacity,
        ).add_to(map_)
    return map_

In [4]:
start_loc = [55.381688072192105, 10.395787567836594]
end_loc = [55.38649345380863, 10.402921484784832]

geohash0 = ghh.encode(
    start_loc[1], start_loc[0], 
    precision = precision,
    bits_per_char = bits_per_char,
)
geohash1 = ghh.encode(
    end_loc[1], end_loc[0], 
    precision = precision,
    bits_per_char = bits_per_char,
)

nseq1, _ = ip.NaiveGeohashSequenceInterpolator._interpolate_geohash_sequence(
    sequence = [geohash0, geohash1],
    distance_delta = 0,
    precision = precision,
    bits_per_char = bits_per_char,
)

nseq2, _ = ip.GeohashSequenceInterpolator._interpolate_geohash_sequence(
    sequence = [geohash0, geohash1],
    distance_delta = 0,
    precision = precision,
    bits_per_char = bits_per_char,
)

print(len(nseq1), len(nseq2))

34 50


In [5]:
map_ = plotutils.setup_map(center)
map_ = plot_sequence(nseq1, 2, "cyan", map_ = map_)
map_

In [6]:
map_ = plotutils.setup_map(center)
map_ = plot_sequence(nseq2, 2, "cyan", map_ = map_)
map_